# Taller 1 - DATASETS Y TOKENIZADORES

Realizado por:
- Ortiz Rodríguez Nicol valeria
- Tovar Parra Carlos Mauricio
- Camargo Arcos Andres David


## Primera Parte (40 puntos)

In [1]:
!pip install datasets spacy

In [2]:
#Modelo pequeño para el procesamiento de lenguaje en español.
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.9 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/12.9 MB 1.8 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/12.9 MB 1.6 MB/s eta 0:00:08
     ---- ----------------------------------- 1.6/12.9 MB 1.7 MB/s eta 0:00:07
     ----- ---------------------------------- 1.8/12.9 MB 1.7 MB/s eta 0:00:07
     ------- -------------------------------- 2.4/12.9 MB 1.7 MB/s eta 0:00:07
     -------- ------------------------------- 2.6/12.9 MB 1.8 MB/s eta 0:00:06
     -------- ------------------------------- 2.9/12.9 MB 1.6 MB/s eta 0:00:07
     -------- ------------------------------- 2.9/12.9 MB 1.6 MB/s eta 0:00:07
     ---------- ----------------------------- 3.4/12.9 MB 1.6 MB/s eta 0:00:07
     ----------- ---------------------------- 3.7/12.9 MB 1.5 MB/s

In [17]:
from datasets import load_dataset
import spacy
from tqdm import tqdm #Para mostrar barra de progreso.
import string
import os
import re

In [6]:
raw_text = load_dataset('large_spanish_corpus', name='DGT', split='train')

raw.zip:  39%|###9      | 2.29G/5.79G [00:00<?, ?B/s]

c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nicol\.cache\huggingface\hub\datasets--large_spanish_corpus. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generating train split:   0%|          | 0/3168368 [00:00<?, ? examples/s]

In [7]:
raw_text

Dataset({
    features: ['text'],
    num_rows: 3168368
})

In [8]:
nlp = spacy.load("es_core_news_sm")

## Generated Tokens

In [9]:
def save_raw_text_with_nlp(raw_text, output_dir, lines_per_file=1000):
    """
    Procesa un conjunto de textos utilizando spaCy y guarda los tokens en archivos divididos por partes.

    Parámetros:
    raw_text (iterable): Un conjunto de datos que contiene registros con textos,
                         donde cada registro es un diccionario que debe incluir la clave 'text'.
    output_dir (str): Directorio donde se guardarán los archivos de salida.
    lines_per_file (int): Número de líneas por archivo de salida.

    Salida:
    - Archivos de texto que contienen los tokens procesados, divididos por partes.
    """
    os.makedirs(output_dir, exist_ok=True)  # Crear directorio si no existe
    all_tokens = []  # Lista para almacenar los tokens de un archivo actual
    file_count = 0  # Contador de archivos

    for i, record in enumerate(tqdm(raw_text, desc="Procesando textos")):
        doc = nlp(record['text'])
        tokens = [token.text for token in doc]  # Extraer tokens
        all_tokens.append(tokens)  # Almacenar tokens en la lista

        # Si alcanzamos el número especificado de líneas, guardar en un archivo
        if (i + 1) % lines_per_file == 0:
            file_name = os.path.join(output_dir, f'tokens_part_{file_count}.txt')
            with open(file_name, 'w', encoding='utf-8') as f:
                for tokens_list in all_tokens:
                    f.write(" ".join(tokens_list) + '\n')
            file_count += 1  # Incrementar contador de archivos
            all_tokens = []  # Reiniciar la lista para el siguiente archivo

    # Guardar cualquier tokens restantes
    if all_tokens:
        file_name = os.path.join(output_dir, f'tokens_part_{file_count}.txt')
        with open(file_name, 'w', encoding='utf-8') as f:
            for tokens_list in all_tokens:
                f.write(" ".join(tokens_list) + '\n')

In [10]:
#path = '/content/drive/MyDrive/Academico/PLN/taller1/output_tokens'
#save_raw_text_with_nlp(raw_text, path, lines_per_file=3000)
save_raw_text_with_nlp(raw_text, "output_tokens_files", lines_per_file=5000)

Procesando textos: 100%|██████████| 3168368/3168368 [5:01:54<00:00, 174.91it/s]    


## Clean Tokens

In [11]:
def create_output_folder(output_folder):
    """Crea la carpeta de salida si no existe."""
    os.makedirs(output_folder, exist_ok=True)

In [12]:
def clean_token(token):
    """Limpia un token según los criterios establecidos."""
    # Eliminamos dígitos
    w = re.sub(r'[0-9]', '', token)  # Depurar de dígitos

    # Eliminamos signos de puntuación
    re_punc = re.compile('[%s]' % re.escape(string.punctuation)) 
    w = re_punc.sub('', w)  # Depurar de signos de puntuación
    
    # Eliminamos símbolos especiales
    w = re.sub(r"\!|\'|\?|\¿|\¡|\«|\»|\—|◦", "", w)  # Depurar de símbolos especiales

    # Eliminamos palabras de una sola letra
    if len(w) == 1:
        return None
        
    # Comprobamos si la cadena resultante no está vacía
    if w.strip():
        # Eliminamos stopwords en español usando spaCy
        if w.lower() in nlp.Defaults.stop_words:
            return None
        return w
    return None

In [13]:
def clean_line(line):
    """Limpia una línea y devuelve una lista de tokens limpios."""
    # Dividir la línea en tokens (asumiendo que ya están separados por espacios)
    tokens = line.strip().split()
    return [clean_token(token) for token in tokens if clean_token(token) is not None]

In [24]:
def process_file(input_file_path, output_file_path):
    """Procesa un archivo y guarda los tokens limpios en el archivo de salida."""
    with open(input_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    cleaned_token_lists = []
    
    # Iteramos sobre cada línea del archivo
    for line in lines:
        cleaned_list = clean_line(line)
        
        # Solo añadimos la lista de tokens si no está vacía
        if cleaned_list:  
            cleaned_token_lists.append(cleaned_list)

    # Guardamos el resultado en el archivo de salida
    with open(output_file_path, 'w', encoding='utf-8') as out_f:
        for tokens_list in cleaned_token_lists:
            out_f.write(" ".join(tokens_list) + '\n')
    
    return cleaned_token_lists

In [25]:
def clean_tokens_files_from_folder(input_folder, output_folder):
    """
    Limpia los tokens de varios archivos en una carpeta según los siguientes criterios:
    1. Elimina números o dígitos.
    2. Elimina stopwords en español.
    3. Elimina puntuación y símbolos especiales.
    4. Elimina espacios en blanco.
    5. Elimina tokens de una sola letra
    
    Los archivos limpios se guardarán en una nueva carpeta y se devuelve una lista de listas con los tokens limpios.

    Parámetros:
    input_folder (str): Carpeta con los archivos de texto a limpiar.
    output_folder (str): Carpeta donde se guardarán los archivos con los tokens limpios.

    Retorna:
    list: Lista de listas con los tokens limpios.
    """
    create_output_folder(output_folder)

    all_cleaned_token_lists = []  # Aquí almacenaremos todas las listas de tokens limpias

    # Iteramos sobre todos los archivos en la carpeta de entrada
    for file_name in tqdm(os.listdir(input_folder), desc="Procesando archivos"):
        input_file_path = os.path.join(input_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name)

        cleaned_token_lists = process_file(input_file_path, output_file_path)
        all_cleaned_token_lists.extend(cleaned_token_lists)

    print(f"Procesamiento completado. Archivos limpios guardados en {output_folder}")

    return all_cleaned_token_lists

In [26]:
sentences_lists = clean_tokens_files_from_folder('output_tokens_files', 'output_clean_tokens_files')

Procesando archivos: 100%|██████████| 634/634 [05:48<00:00,  1.82it/s]


Procesamiento completado. Archivos limpios guardados en output_clean_tokens_files


## Primeras 100 sentencias

In [23]:
print("[", end="")  
for idx, line in enumerate(sentences_lists[:100]):
    if idx == 0:
        print(f"{line},")  
    else:
        print(f" {line},")  
print("]")  

[['Reglamento', 'Ejecución', 'UE', 'Comisión'],
 ['noviembre'],
 ['establecen', 'valores', 'importación', 'alzado', 'determinación', 'precio', 'entrada', 'determinadas', 'frutas', 'hortalizas'],
 ['COMISIÓN', 'EUROPEA'],
 ['Visto', 'Tratado', 'Funcionamiento', 'Unión', 'Europea'],
 ['Visto', 'Reglamento', 'CE', 'Consejo', 'octubre', 'crea', 'organización', 'común', 'mercados', 'agrícolas', 'establecen', 'disposiciones', 'específicas', 'determinados', 'productos', 'agrícolas', 'Reglamento', 'único', 'OCM'],
 ['Visto', 'Reglamento', 'Ejecución', 'UE', 'Comisión', 'junio', 'establecen', 'disposiciones', 'aplicación', 'Reglamento', 'CE', 'Consejo', 'sectores', 'frutas', 'hortalizas', 'frutas', 'hortalizas', 'transformadas', 'particular', 'artículo', 'apartado'],
 ['Considerando'],
 ['Reglamento', 'Ejecución', 'UE', 'establece', 'aplicación', 'resultados', 'negociaciones', 'comerciales', 'multilaterales', 'Ronda', 'Uruguay', 'criterios', 'Comisión', 'fije', 'valores', 'importación', 'alzado